Bird Detection with Open CV

Source:https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/

In [50]:
# import the necessary packages
import numpy as np,pandas as pd
import argparse
import cv2
import glob
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt

In [52]:
print("[INFO] loading datasets...")
Birds=pd.read_csv('AllData.csv')
ImagePaths=Birds['Image Path'].values
Sizes=Birds['Image Size'].str.replace('(','').str.replace(')','').str.split(',').values
Sizes=[[int(float(j)) for j in i] for i in Sizes]
Labels=Birds['Bird Class'].values
Num_Images=ImagePaths.shape[0]
try:
    shutil.rmtree('Cropped')
    os.mkdir('Cropped')
except:
    os.mkdir('Cropped')
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt.txt" , "MobileNetSSD_deploy.caffemodel" )

[INFO] loading datasets...
[INFO] loading model...


In [43]:
def VisualiseImage(index):
    img = cv2.imread(ImagePaths[index])
    img2 = img[:,:,::-1]
    plt.imshow(img)

In [31]:
#Num_Images=60
count=0
DetectionStats=pd.DataFrame(columns=['path to original','path to crop','startX', 'startY', 'endX', 'endY','confidence','width','height','class'])
for image_indices in np.array_split(range(Num_Images),(Num_Images/500)+1):
    Images = [cv2.imread(Path) for Path in ImagePaths[image_indices]]
    images=[cv2.resize(image,(300,300)) for image in Images]
    images=np.stack(images,axis=0)
    blob=cv2.dnn.blobFromImages(images, 0.007843, (300, 300), 127.5)
    blob_shape=blob.shape
    print("{}:{}  Dealing with {} images of sizes {}".format(image_indices[0],image_indices[-1],blob_shape[0],blob.shape[1:]))
    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()
    detections=detections.reshape(detections.shape[2],detections.shape[3])[[detections[0,0,:,1]==3]]
    print("Detected {} birds".format(detections.shape[0]))
    for i in np.arange(0, detections.shape[0]):
        count+=1
        confidence = detections[i, 2]
        ImageIndex=  image_indices[0]+int(detections[i, 0])
        w=Sizes[ImageIndex][1]    
        h=Sizes[ImageIndex][0] 
        path_to_crop="Cropped/Bird{}_{}.jpg".format(ImageIndex,count)
        path_to_original=ImagePaths[ImageIndex]
        box = detections[i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        label=Labels[ImageIndex]
        DetectionStats.loc[count]=[path_to_original,path_to_crop,startX, startY, endX, endY,confidence,w,h,label]
        crop_img = Images[ImageIndex-image_indices[0]][startY:endY, startX:endX,:]
        cv2.imwrite(path_to_crop, crop_img)

0:19  Dealing with 20 images of sizes (3, 300, 300)
Detected 20 birds
20:39  Dealing with 20 images of sizes (3, 300, 300)
Detected 25 birds
40:59  Dealing with 20 images of sizes (3, 300, 300)
Detected 21 birds


In [46]:
DetectionStats.to_csv("DetectionStats.csv")

In [33]:
print("Overall Statistics:")
print("No of Images={}".format(len(ImagePaths)))
print("No of Images with Detected Birds={}".format(np.unique(DetectionStats['path to original']).shape[0]))
print("Total No of Birds Detected={}".format(DetectionStats.shape[0]))

Overall Statistics:
No of Images=15164
No of Images with Detected Birds=57
Total No of Birds Detected=66
